<a href="https://colab.research.google.com/github/TurkuNLP/DIGHT25/blob/main/01_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DSPy tutorial part 1:

*   Grabs a dataset on climate change claims
*   Carries out the same classification task using DSPy
*   Compares the outputs

In [1]:
!pip3 install -q 'datasets<4.0.0'
!pip3 install -q dspy

#Get the API key
!wget -O api-key.txt http://epsilon-it.utu.fi/dight-api-key-1.txt
api_key=open("api-key.txt").read().strip()

#Backup option:
#api_key="sk_...."

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.4/247.4 kB 20.5 MB/s eta 0:00:00
--2025-08-28 08:08:32--  http://epsilon-it.utu.fi/dight-api-key-1.txt
Resolving epsilon-it.utu.fi (epsilon-it.utu.fi)... 130.232.253.13
Connecting to epsilon-it.utu.fi (epsilon-it.utu.fi)|130.232.253.13|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 165 [text/plain]
Saving to: ‘api-key.txt’

api-key.txt         100%[===================>]

In [2]:
from datasets import load_dataset
from google.colab import userdata
import random


#### new comment

# Load the Frugal AI Challenge dataset
dataset = load_dataset("QuotaClimat/frugalaichallenge-text-train")

# Access the training split
train_data = dataset["train"]

# Pick 30 random items
sampled_items = random.sample(list(train_data), 30)

# Print out a few samples to see the structure
for item in sampled_items[:10]: #prints 10 samples
    print(" Quote:", item["quote"])
    print(" Label:", item["label"])
    print()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.02M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/248k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1219 [00:00<?, ? examples/s]

 Quote: My overall position on global warming is that, as a physicist — I studied physics at Cambridge — I am not one of those who deny that carbon dioxide emissions heat the planet. They do have that effect, although there is less certainty about how much the complex feedback effects that climate models seek to replicate may amplify the comparatively modest effect of increasing  and other greenhouse gas emissions. Nonetheless, in my view it is wise to take measures to prevent and to adapt to global warming, and it is sensible to try to assess the costs and benefits of action and inaction to ensure that we adopt the most cost-effective approach. The simple fact is that, since the beginning of this century, the average global temperature has flatlined; indeed, over the past 18 months it has fallen back, and according to the satellite measurements of temperature, it is now basically back at the level it was in 1979, when such measurements started to be taken.
 Label: 5_science_unreliable

In [3]:
import dspy


class ClimateViewClassifier(dspy.Signature):
    """Classify a quote or claim about climate change using one of the following labels:

    Not-Relevant: No relevant claim detected or claims that don't fit other categories

    Not-Happening: Claims denying the occurrence of global warming and its effects - Global warming is not happening. Climate change is NOT leading to melting ice (such as glaciers, sea ice, and permafrost), increased extreme weather, or rising sea levels. Cold weather also shows that climate change is not happening

    Not-Human: Claims denying human responsibility in climate change - Greenhouse gases from humans are not the causing climate change.

    Not-Bad: Claims minimizing or denying negative impacts of climate change - The impacts of climate change will not be bad and might even be beneficial.

    Solutions-Harmful-Unnecessary: Claims against climate solutions - Climate solutions are harmful or unnecessary

    Science-is-Unreliable: Claims questioning climate science validity - Climate science is uncertain, unsound, unreliable, or biased.

    Proponents-Biased: Claims attacking climate scientists and activists - Climate scientists and proponents of climate action are alarmist, biased, wrong, hypocritical, corrupt, and/or politically motivated.

    Fossil-Fuels-Needed: Claims promoting fossil fuel necessity - We need fossil fuels for economic growth, prosperity, and to maintain our standard of living.
    """

    quote = dspy.InputField(desc="Quote or claim about climate change")
    label = dspy.OutputField(
        desc="Label of the quote",
        choices=["Not-Happening","Not-Human","Not-Bad","Solutions-Harmful-Unnecessary","Science-is-Unreliable","Proponents-Biased","Fossil-Fuels-Needed"],
    )

# Initialize DSPy with an LLM (use default OpenAI if configured, or HF local)
lm=dspy.LM("openai/gpt-4.1-mini",api_key=api_key)
dspy.configure(lm=lm)

# Instantiate classifier
classifier = dspy.Predict(signature=ClimateViewClassifier)

# Run predictions on our 30 samples
for item in sampled_items:
    quote = item["quote"]
    true_label = item["label"]
    pred = classifier(quote=quote)
    print("Text:", quote.replace("\n", " "))
    print("True:", true_label)
    print("Pred:", pred.label)
    print("-" * 40)


Text: My overall position on global warming is that, as a physicist — I studied physics at Cambridge — I am not one of those who deny that carbon dioxide emissions heat the planet. They do have that effect, although there is less certainty about how much the complex feedback effects that climate models seek to replicate may amplify the comparatively modest effect of increasing  and other greenhouse gas emissions. Nonetheless, in my view it is wise to take measures to prevent and to adapt to global warming, and it is sensible to try to assess the costs and benefits of action and inaction to ensure that we adopt the most cost-effective approach. The simple fact is that, since the beginning of this century, the average global temperature has flatlined; indeed, over the past 18 months it has fallen back, and according to the satellite measurements of temperature, it is now basically back at the level it was in 1979, when such measurements started to be taken.
True: 5_science_unreliable
Pre

# Follow-up task(s)

1.   Is the model accurate?
1.   Is the model wrong, when it disagrees with the dataset annotation?
1.   If you change the model from gpt-4.1-mini to gpt-4.1 is there any difference? (do not rerun the whole notebook so you keep your random sample)
2.   Think of any other classification schema of these claims, and try to modify the DSPy code to make it happen - did it work?
